In [1]:
import os, joblib
import numpy as np
import pandas as pd
from spark.preprocessor import load_q_data, load_timeseries_data, train_test_split_ids, Preprocess_Q
from spark.model_io import load_model
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, balanced_accuracy_score
from sklearn.model_selection import cross_validate,GridSearchCV
from sklearn.model_selection import train_test_split, StratifiedKFold

## Load Data

In [2]:
#Data file directory
raw_prep_dir  = '../processed_data/'

In [3]:
#Load questionair data
X_q_data, y_data, id_list = load_q_data(raw_prep_dir + 'merged_dfq.csv')

In [4]:
X_train_id, X_test_id, y_train, y_test = train_test_split_ids (id_list, y_data, test_size = 0.2, stratify = True)
print(X_train_id.shape, X_test_id.shape)

(375,) (94,)


In [5]:
X_train_q = X_q_data[X_q_data.id.isin(X_train_id)]
X_test_q = X_q_data[X_q_data.id.isin(X_test_id)]
print(X_train_q.shape)

(375, 39)


In [6]:
prep = Preprocess_Q(feature_importance = True)
prep.fit(X_train_q)
X_q_train_prep = prep.transform(X_train_q)
X_q_test_prep = prep.transform(X_test_q)
pd.DataFrame(X_q_train_prep)

,0,1,2,3,4,5,6,7,8,9
0,-0.166667,-0.5625,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.555556,1.0000,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
2,0.333333,0.6250,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,0.055556,0.4375,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
4,0.777778,0.5625,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
370,0.166667,0.0000,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
371,1.388889,1.1875,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
372,-0.222222,-0.5000,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
373,0.777778,0.6875,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0


## Load Model

In [7]:
model = load_model("/Users/sebastian.zilles/code/marzecovaa/spark/model/model_20250908-155122.joblib")

Loading model from: /Users/sebastian.zilles/code/marzecovaa/spark/model/model_20250908-155122.joblib


In [8]:
model

,estimators,"[('svc', ...), ('catboost', ...)]"
,final_estimator,LogisticRegre...ndom_state=42)
,cv,StratifiedKFo... shuffle=True)
,stack_method,'auto'
,n_jobs,-1
,passthrough,True
,verbose,0
,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'


In [9]:
model.fit(X_q_train_prep, y_train)

/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: invalid value encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:336: RuntimeWarning: divide by zero encountered in matmul
  grad[:, :n_features] = grad_pointwise.T @ X + 

,estimators,"[('svc', ...), ('catboost', ...)]"
,final_estimator,LogisticRegre...ndom_state=42)
,cv,StratifiedKFo... shuffle=True)
,stack_method,'auto'
,n_jobs,-1
,passthrough,True
,verbose,0
,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'


In [10]:
y_pred_pca_combo = model.predict(X_q_test_prep)
balanced_accuracy_score(y_test, y_pred_pca_combo)

0.4478590250329381

In [ ]:
cv_scores = cross_validate(
    model, X_q_train_prep, y_train,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=True),
    scoring='balanced_accuracy'
)

/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:203: RuntimeWarning: invalid value encountered in matmul
  raw_prediction = X @ weights.T + intercept  # ndarray, likely C-contiguous
/Users/sebastian.zilles/.pyenv/versions/3.10.6/envs/spark/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:336: RuntimeWarning: divide by zero encountered in matmul
  grad[:, :n_features] = grad_pointwise.T @ X + 

In [17]:
score = np.mean(cv_scores["test_score"])
score

np.float64(0.3342681879523985)